# Packages and Overall Info

In [1]:
%reload_kedro

[03/03/23 14:21:18] INFO     Resolved project path as:                                              ]8;id=938768;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=711604;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py#132\132]8;;\
                             /home/work/Documents/repo/economic_freedom_ml.                                        
                             To set a different path, run '%reload_kedro <project_root>'                           

[03/03/23 14:21:19] INFO     Kedro project economic_freedom_ml                                      ]8;id=896960;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=395921;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py#101\101]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=319196;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=988993;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py#102\102]8;;\
                             'pipelines'                                                                           

In [20]:
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from prophet import Prophet
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from pandas_profiling import ProfileReport

In [3]:
model_input = catalog.load("data_with_features")

[03/03/23 14:21:27] INFO     Loading data from 'data_with_features' (CSVDataSet)...             ]8;id=423722;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=720552;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [4]:
model_input.head()

,year,score,property_right,government_integrity,tax_burden,government_spending,business_freedom,labor_freedom,monetary_freedom,trade_freedom,...,peru,saint_lucia,saint_vincent_and_the_grenadines,suriname,trinidad_and_tobago,united_states,uruguay,venezuela,north_america,south_america
0,2023,50.1,35.1,45.1,73.3,53.0,55.1,51.0,37.9,60.6,...,0,0,0,0,0,0,0,0,0,1
1,2023,71.3,72.6,68.7,80.6,70.8,64.7,63.4,78.6,58.4,...,0,0,0,0,0,0,0,0,1,0
2,2023,56.6,34.7,38.1,77.0,64.1,54.0,59.9,82.5,55.6,...,0,0,0,0,0,0,0,0,1,0
3,2023,43.0,14.1,28.7,86.0,58.8,54.7,46.2,72.8,60.8,...,0,0,0,0,0,0,0,0,0,1
4,2023,53.3,50.3,40.0,69.9,53.8,63.2,55.9,78.4,60.0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
model_input_info = pd.DataFrame({'column': model_input.columns,  
                             'dtype': model_input.dtypes,
                             'missing_percent': model_input.isna().sum()/model_input.shape[0],
                             'count': model_input.count(),
                             'n_unique': model_input.nunique()}).reset_index().drop('index', axis=1).round(decimals = 3),  
print(model_input_info)

(                              column    dtype  missing_percent  count  \
0                               year    int64              0.0    883   
1                              score  float64              0.0    883   
2                     property_right  float64              0.0    883   
3               government_integrity  float64              0.0    883   
4                         tax_burden  float64              0.0    883   
5                government_spending  float64              0.0    883   
6                   business_freedom  float64              0.0    883   
7                      labor_freedom  float64              0.0    883   
8                   monetary_freedom  float64              0.0    883   
9                      trade_freedom  float64              0.0    883   
10                investment_freedom  float64              0.0    883   
11                 financial_freedom  float64              0.0    883   
12                         argentina    int64     

In [6]:
model_input[model_input.isna().any(axis=1)].head()

,year,score,property_right,government_integrity,tax_burden,government_spending,business_freedom,labor_freedom,monetary_freedom,trade_freedom,...,peru,saint_lucia,saint_vincent_and_the_grenadines,suriname,trinidad_and_tobago,united_states,uruguay,venezuela,north_america,south_america


# Data Science

## Preparing the Data

### Separating the data into feature data and target data (X_all and y_all, respectively)

In [7]:
model_input = model_input.set_index("year")

In [8]:
X_all = model_input.drop('score', axis=1)
y_all = model_input['score']

In [9]:
X_all.head()

,property_right,government_integrity,tax_burden,government_spending,business_freedom,labor_freedom,monetary_freedom,trade_freedom,investment_freedom,financial_freedom,...,peru,saint_lucia,saint_vincent_and_the_grenadines,suriname,trinidad_and_tobago,united_states,uruguay,venezuela,north_america,south_america
year,,,,,,,,,,,,,,,,,,,,,
2023,35.1,45.1,73.3,53.0,55.1,51.0,37.9,60.6,55.0,60.0,...,0,0,0,0,0,0,0,0,0,1
2023,72.6,68.7,80.6,70.8,64.7,63.4,78.6,58.4,70.0,60.0,...,0,0,0,0,0,0,0,0,1,0
2023,34.7,38.1,77.0,64.1,54.0,59.9,82.5,55.6,55.0,50.0,...,0,0,0,0,0,0,0,0,1,0
2023,14.1,28.7,86.0,58.8,54.7,46.2,72.8,60.8,15.0,40.0,...,0,0,0,0,0,0,0,0,0,1
2023,50.3,40.0,69.9,53.8,63.2,55.9,78.4,60.0,60.0,50.0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
y_all.head()

year
2023    50.1
2023    71.3
2023    56.6
2023    43.0
2023    53.3
Name: score, dtype: float64

### Training and Testing Data Split

In [11]:
X_train = X_all[X_all.index<2019]
y_train = y_all[y_all.index<2019]

X_test = X_all[X_all.index>=2019]
y_test = y_all[y_all.index>=2019]

print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 723 samples.
Testing set has 160 samples.


## Random Forest 

In [12]:
random_forest = RandomForestRegressor(random_state=999)
random_forest.fit(X_train, y_train)
rf_predicted = random_forest.predict(X_test)

In [15]:
print(r2_score(y_test,rf_predicted))

0.9293079985828352


## Prophet (Meta)

## XGBoost